# layerwise relevance propagation

主要参考以下资料简单了解LRP的基本概念。

- [InDepth: Layer-Wise Relevance Propagation](https://towardsdatascience.com/indepth-layer-wise-relevance-propagation-340f95deb1ea)
- [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007%2F978-3-030-28954-6_10)
- [www.heatmapping.org](http://www.heatmapping.org/)
- [Why&How: Interpretable ML](https://towardsdatascience.com/why-how-interpretable-ml-7288c5aa55e4)
- [Opening the black box of LSTM models using XAI](https://agu.confex.com/agu/fm20/videogateway.cgi/id/29034?recordingid=29034)

分层相关性传播 (LRP) 是可解释机器学习 (XML) 中最突出的方法之一。这里简单记录 LRP 的细节和实现它的一些技巧。

LRP的目的是提供一个对任何神经网络输出在其输入域内的解释，即对每个预测，每个输出的贡献有多少；每个输入对输出总的贡献有多少等。比如网络根据乳房 X 光照片（乳房组织的图像）预测癌症诊断，那么 LRP 给出的解释将是展示原始图像中哪些像素有助于诊断以及在多大程度上有助于诊断的图。此方法不与网络的训练交互，因此可以轻松地将其应用于已训练的分类器。

XML 方法在安全很关键领域特别有用，在这些领域中，从业者必须确切地知道网络正在关注什么。其他用例包括网络（错误）行为的诊断、科学发现和改进网络架构。

## 方法介绍

直观地说，LRP 所做的是，它使用网络权重和由前向传递创建的神经激活将输出传播回网络，直到输入层。在那里，我们可以可视化哪些像素真正对输出做出了贡献。我们将每个像素或中间神经元的贡献大小称为“相关性”值R。

LRP 是一种守恒的技术，这意味着任何输出y的大小通过反向传播过程都是守恒的，并且等于输入层的相关图R的总和。此属性适用于任何连续的层j和k，以及输入和输出层的传递性。

一步一步来。假设网络是一个分类器，它的输出是一个大小为M的向量，其中每一项对应于M 个类之一。在输出层，我们选择一个我们想要解释的神经元或类。对于这个神经元，相关性等于它的激活，输出层中所有其他神经元的相关性为零。举例来说，如果我们要使用LRP找出网络中的神经元和输入对于预测类ç的相关性，我们先从输出神经元类ç开始，看看网络如何到达这个神经元的值。

然后，我们按照这个基本的 LPR 规则在网络中倒行：

![](img/1_yAGiGS8nHfXxrnXP_u_P8Q.png)LRP-0

这里，j和k是任意连续层的两个神经元。我们已经知道输出层中的相关性R，所以我们将从那里开始，并迭代地使用这个公式来计算前一层的每个神经元的R。a表示各个神经元的激活，w是两个神经元之间的权重。

这是最简单的 LRP 规则。根据应用程序，可能希望使用不同的规则，这将在后面讨论。它们都遵循相同的基本原则。

现在，这个公式是什么意思？分子是神经元j对神经元k 的影响量。为了加强上面的守恒性质，必须除以下层所有神经元的贡献总和。我们通常在分母上添加一个非常小的常数 ϵ 以使分数更稳定。k 上的外部总和意味着神经元j的相关性由其对下一层所有神经元k的影响之和乘以这些神经元的相关性决定。

我们可以简单地从最后一层直到第一层。根据我们选择从哪个输出神经元开始，我们可以获得任何类别的解释。因此，我们甚至可以检查网络认为与预测任何A类相关的内容，即使它实际上预测了B类！

没有特别限制可以这样分析的网络结构，LRP 适用于 CNN，也可以用于 LSTM。但是，要求网络仅包含 ReLU 激活函数。

为了更好地了解应用此公式时正在进行的过程，将其解释为 4 步过程。

![](img/1_sA5XCLOMwKnmH-l7H-aHCg.png)

![](img/0_Fg0u4MmcQ0lu3S0C.png)

第一步确定上层每个神经元影响的总和，类似于一个修改版的前向传播。请注意，对于 ReLU 层，它与正常前向传递相同，除了我们添加一个小的常数 ϵ 并在我们的权重周围包裹一个函数 ρ。这个 ρ 只是为了使公式更通用，并包含我们稍后将看到的所有可能的 LRP 规则。在我们之前的情况下，ρ 将是恒等函数，这意味着我们可以忽略它。

请注意，总和经过了下层中的每个神经元 j，也经过了偏置神经元。在接下来的所有步骤中，偏置将被忽略，因为我们希望相关性仅流向输入神经元，而不是最终进入静态偏置神经元。

在第二步中，我们简单地将每个神经元在更高层中的相关性除以之前计算的z值。这确保了守恒性质成立。

在第三步中，我们为前一层中的每个神经元计算一个数量c，因此可以将其视为反向传播。这个c可以粗略地看作是有多少相关性从后续层向下传递到神经元j。

最后，在第四步中，将来自上面的相关性与神经元的激活相乘以计算其自身的相关性。直观地说，一个神经元是相关的，如果 1) 它具有高激活，并且 2) 它对更高层的相关神经元贡献很大。

现在应该已经能够自己实现 LRP 了！如果只想开箱即用地使用它，请注意已经有[出色的实现](https://github.com/albermax/innvestigate)

## 实现技巧

首次介绍时，LRP 的作者提供了一个伪代码片段，该片段将通过网络逐层计算相关性。（查看这个[简短的教程](https://git.tu-berlin.de/gmontavon/lrp-tutorial)来实现这个基本版本）随着他们自己应用 LRP 的经验越来越丰富，他们发布了一种更有效的方法来实现它。

关键部分是利用高效的 autograd 框架，例如 pytorch 或 tensorflow 为我们进行反向传递。

![](img/0_ZecaNYsvUsyMIHyq.png)

与我们上面看到的公式的最大区别在于，我们将c值计算为梯度计算。

![](img/1_GEa_yDV7FRkTOizyYX6JEA.png)

这使我们能够利用高效的自动反向传播运算。为了解决这个问题并给出正确的结果，我们必须将 s 视为一个常数。这由上面代码中的 .data 指示。

要注意 LRP 有不同的传播规则，可能希望将其中的几个组合起来以获得最佳结果。幸运的是，[这里](https://link.springer.com/chapter/10.1007%2F978-3-030-28954-6_10)有一个最重要规则的整洁列表，并附有关于何时使用它们的建议。

输入层：对于图像，LRP 的作者在Deep Taylor Decomposition 论文中引入了选择规则，并采用以下形式：

![](img/1_4GN3n2GnCGWUCezSCr1jbA.png)LRP-z

这里l和h分别是允许的最低和最高像素值。

低层：这里我们希望解释更流畅，噪音更小，因为这些层已经非常接近我们人类将看到并且必须理解的相关图。为此，我们可以使用 LRP-γ 规则，即不成比例地支持正面证据而不是负面证据：

![](img/1_QjMMcc7XuEZMdYuHYWFD9A.png)LRP-γ

高层：上面的 LRP-γ 或以下 LRP-ϵ 规则在这里都可以很好地工作。他们从相关图中去除了一些噪音。特别是，ϵ 会吸收一些小的或矛盾的证据。

![](img/1_RtMBnnE6fnTAEEWk4XNhQQ.png)LRP-ϵ

输出层：这里我们理想地使用未修改的 LRP-0 规则。

还有比这些更多的规则，但是此处看到的规则足以满足大多数应用程序的要求。如果选择正确的规则过于复杂，请不要担心！仅仅 LRP-0 和 LRP-ϵ 应该就足够了，一旦得到了符合这些规则的解释，总是可以决定是否花时间试验其他规则并使解释更漂亮。为了对不同的规则有一点直观的了解，建议玩一下[交互式演示](https://lrpserver.hhi.fraunhofer.de/image-classification)。